In [ ]:
import json
from gcms_spectra_gnn.datasets import MoleculeJSONDataset
from gcms_spectra_gnn.preprocess import Preprocessor
from gcms_spectra_gnn.backend import JSONDirectoryBackend
import numpy as np
import os
from functools import partial

In [ ]:
mgf_json_file = '../../data/NIST_GC_EI_MAX_LIBRARY.json'

In [ ]:
with open(mgf_json_file) as fh:
    library = json.load(fh)

In [ ]:
len(library)

In [ ]:
# set up bakcend directory
lib_path = '../../data/preprocessed_full_21-03-01'
if not os.path.exists(lib_path):
    os.makedirs(lib_path)

lib_index = os.path.join(lib_path, 'index.json')
if not os.path.exists(lib_index):
    with open(lib_index, 'w') as fh:
        json.dump(library, fh)

In [ ]:
backend = JSONDirectoryBackend(lib_index)

In [ ]:
prefix = ''

BEGIN_IDX = 0
END_IDX = len(library)

def slicer(x):
    return x[BEGIN_IDX:END_IDX]

spectra = [[entry['M/Z'], entry['INTENSITY']] for entry in slicer(library) if 'SMILES' in entry]
smiles = [entry['SMILES'] for entry in slicer(library) if 'SMILES' in entry]
indices = [int(entry['SCANS']) for entry in slicer(library) if 'SMILES' in entry]

preproc = Preprocessor(backend, add_h=True)

FLUSH_FREQ = 10000
def flush_hook(i, preprocessor, freq=FLUSH_FREQ):
    if (i % freq == 0) and (i > 0):
        print('flushing')
        preprocessor.backend.flush()
        print('length:', len(preprocessor.backend))

preproc.transform(spectra, smiles, indices, hook=partial(flush_hook, freq=FLUSH_FREQ))
backend.flush()

In [ ]:
backend.flush()